<a href="https://colab.research.google.com/github/FranciscoOcampoPredictiva/azureml_course/blob/main/Lecture_5_Logistic_Regression_in_Local_Environment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Part 1: Data Processing

## Importing the libraries and dataset



In [3]:
import numpy as np
import pandas as pd

In [4]:
# Loading the dataset
dataset = pd.read_csv('/content/Churn_Modelling.csv')

In [5]:
dataset.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [6]:
dataset.shape

(10000, 14)

In [7]:
dataset.columns

Index(['RowNumber', 'CustomerId', 'Surname', 'CreditScore', 'Geography',
       'Gender', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard',
       'IsActiveMember', 'EstimatedSalary', 'Exited'],
      dtype='object')

## Dealing with the missing data

In [8]:
dataset.isnull().values.any()

False

## Encoding the categorical variables

In [9]:
dataset.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [10]:
dataset = dataset.drop(columns=['RowNumber', 'CustomerId', 'Surname'])

In [11]:
dataset.shape

(10000, 11)

In [12]:
dataset.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [13]:
dataset.select_dtypes(include='object').columns

Index(['Geography', 'Gender'], dtype='object')

In [14]:
dataset = pd.get_dummies(dataset, drop_first=True)

In [15]:
dataset.head()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_Germany,Geography_Spain,Gender_Male
0,619,42,2,0.00,1,1,1,101348.88,1,0,0,0
1,608,41,1,83807.86,1,0,1,112542.58,0,0,1,0
2,502,42,8,159660.80,3,1,0,113931.57,1,0,0,0
3,699,39,1,0.00,2,0,0,93826.63,0,0,0,0
4,850,43,2,125510.82,1,1,1,79084.10,0,0,1,0


In [16]:
dataset.shape

(10000, 12)

## Splitting the dataset into train and test set

In [17]:
X = dataset.drop(columns='Exited')
Y = dataset['Exited']

In [18]:
from sklearn.model_selection import train_test_split
x_test, x_train, y_test, y_train = train_test_split(X, Y, test_size=0.2, random_state=0)

# Part 2:Training and evaluating the ML Model

In [19]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(random_state=0)
classifier.fit(x_train, y_train)

LogisticRegression(random_state=0)

In [20]:
# predicting the results on test set
y_pred = classifier.predict(x_test)

In [21]:
# predicting the probabilities
y_prob = classifier.predict_proba(x_test)[:, 1]

In [22]:
y_pred.shape

(8000,)

In [23]:
y_prob.shape

(8000,)

In [24]:
# getting the confusion matrix and the accuracy score
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
acc = accuracy_score(y_test, y_pred)

In [25]:
print(cm)
print(acc)

[[6236  132]
 [1543   89]]
0.790625


# Part 3: Creating the final dataset with predicted values and probabilities

In [26]:
x_test = x_test.reset_index(drop=True)
y_test = y_test.reset_index(drop=True)

y_prob_df = pd.DataFrame(y_prob, columns=['Scored probabilities'])
y_pred_df = pd.DataFrame(y_pred, columns=['Scored Lebels'])

# create the final dataset
scored_dataset = pd.concat([x_test, y_test, y_pred_df, y_prob_df], axis=1)

In [27]:
print(scored_dataset)

      CreditScore  Age  Tenure    Balance  NumOfProducts  HasCrCard  \
0             667   34       5       0.00              2          1   
1             427   42       1   75681.52              1          1   
2             535   29       2  112367.34              1          1   
3             654   40       5  105683.63              1          1   
4             850   57       8  126776.30              2          1   
...           ...  ...     ...        ...            ...        ...   
7995          594   32       4  120074.97              2          1   
7996          794   22       4  114440.24              1          1   
7997          738   35       5  161274.05              2          1   
7998          590   38       9       0.00              2          1   
7999          623   48       1  108076.33              1          1   

      IsActiveMember  EstimatedSalary  Geography_Germany  Geography_Spain  \
0                  0        163830.64                  0              